In [ ]:
import sys
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/poc")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/plugins")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/dags")

#Import libs python
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import date

#Import libs internas
from utils import spark_utils_session as utils

from hooks.hdfs.hdfs_helper import HdfsHelper
from jobs.job_base_config import BaseETLJobClass

import poc_helper
poc_helper.load_env("PROD")

In [ ]:
def get_session(profile: str, dynamic_allocation_enabled: bool = True) -> utils.DBASparkAppSession:
    """Generates DBASparkAppSession."""
    
    app_name = "tsevero_gei"
    
    spark_builder = (utils.DBASparkAppSession
                     .builder
                     .setAppName(app_name)
                     .usingProcessProfile(profile)
                    )
    
    if dynamic_allocation_enabled:
        spark_builder.autoResourceManagement()

    return spark_builder.build()

session = get_session(profile='efd_t2')

In [ ]:
session.sparkSession.sql("SHOW DATABASES").show(truncate=False)

In [ ]:
# Bibliotecas para análise e visualização
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (16, 8)
plt.rcParams['font.size'] = 11

# Acesso ao SparkSession
spark = session.sparkSession

print("=" * 80)
print("SISTEMA DE ANÁLISE GEI - Gestão de Empresas e Indícios Fiscais")
print("=" * 80)
print(f"Sessão Spark: {spark.sparkContext.appName}")
print(f"Versão Spark: {spark.version}")
print(f"Iniciado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 80)

In [ ]:
print("\n" + "=" * 80)
print("VERIFICAÇÃO DE TABELAS DISPONÍVEIS")
print("=" * 80)

# Lista de tabelas esperadas
tabelas_esperadas = [
    'gessimples.gei_percent',
    'gessimples.gei_cnpj',
    'gessimples.gei_nfe_completo',
    'gessimples.gei_cadastro',
    'gessimples.gei_contador',
    'gessimples.gei_socios_metricas',
    'gessimples.gei_c115_ranking_risco_grupo_economico',
    'gessimples.gei_indicios_metricas_grupo',
    'gessimples.gei_pagamentos_metricas_grupo',
    'gessimples.gei_funcionarios_metricas_grupo'
]

print("\nVerificando tabelas do sistema GEI:\n")
for tabela in tabelas_esperadas:
    try:
        count = spark.sql(f"SELECT COUNT(*) as cnt FROM {tabela}").collect()[0]['cnt']
        print(f"✓ {tabela:60s} -> {count:,} registros")
    except Exception as e:
        print(f"✗ {tabela:60s} -> NÃO ENCONTRADA")

print("\n" + "=" * 80)

In [ ]:
print("\n" + "=" * 80)
print("1. PANORAMA GERAL DO SISTEMA GEI")
print("=" * 80)

query_panorama = """
SELECT 
    COUNT(DISTINCT num_grupo) AS total_grupos,
    SUM(qntd_cnpj) AS total_cnpjs,
    ROUND(AVG(score_final_avancado), 2) AS score_medio,
    ROUND(MAX(score_final_avancado), 2) AS score_maximo,
    COUNT(CASE WHEN score_final_avancado >= 20 THEN 1 END) AS grupos_criticos,
    COUNT(CASE WHEN score_final_avancado >= 15 AND score_final_avancado < 20 THEN 1 END) AS grupos_alto_risco,
    COUNT(CASE WHEN score_final_avancado >= 10 AND score_final_avancado < 15 THEN 1 END) AS grupos_medio_risco,
    SUM(COALESCE(valor_max, 0)) AS receita_total_monitorada,
    COUNT(CASE WHEN valor_max >= 4800000 THEN 1 END) AS grupos_acima_limite_sn
FROM gessimples.gei_percent
"""

df_panorama = spark.sql(query_panorama)
panorama = df_panorama.collect()[0]

print(f"\nMÉTRICAS PRINCIPAIS:")
print(f"  Total de Grupos Monitorados: {panorama['total_grupos']:,}")
print(f"  Total de CNPJs: {panorama['total_cnpjs']:,}")
print(f"  Score Médio de Risco: {panorama['score_medio']:.2f}")
print(f"  Score Máximo Detectado: {panorama['score_maximo']:.2f}")
print(f"\nDISTRIBUIÇÃO DE RISCO:")
print(f"  Grupos Críticos (≥20): {panorama['grupos_criticos']:,}")
print(f"  Grupos Alto Risco (15-20): {panorama['grupos_alto_risco']:,}")
print(f"  Grupos Médio Risco (10-15): {panorama['grupos_medio_risco']:,}")
print(f"\nIMPACTO FINANCEIRO:")
print(f"  Receita Total Monitorada: R$ {panorama['receita_total_monitorada']:,.2f}")
print(f"  Grupos Acima Limite SN: {panorama['grupos_acima_limite_sn']:,}")

# Gráfico de Distribuição por Faixa de Risco
dist_risco = spark.sql("""
SELECT 
    CASE 
        WHEN score_final_avancado >= 25 THEN 'Crítico Extremo (≥25)'
        WHEN score_final_avancado >= 20 THEN 'Crítico (20-25)'
        WHEN score_final_avancado >= 15 THEN 'Alto (15-20)'
        WHEN score_final_avancado >= 10 THEN 'Médio (10-15)'
        WHEN score_final_avancado >= 5 THEN 'Baixo (5-10)'
        ELSE 'Mínimo (<5)'
    END AS faixa_risco,
    COUNT(*) AS quantidade,
    SUM(COALESCE(valor_max, 0)) AS bc_total
FROM gessimples.gei_percent
GROUP BY 1
ORDER BY MIN(score_final_avancado) DESC
""").toPandas()

dist_risco['bc_total'] = dist_risco['bc_total'].astype(float)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))
fig.suptitle('Distribuição de Grupos por Faixa de Risco', fontsize=16, fontweight='bold')

colors = ['#d62728', '#ff7f0e', '#ffdd70', '#8c564b', '#1f77b4', '#2ca02c']
ax1.pie(dist_risco['quantidade'], labels=dist_risco['faixa_risco'], autopct='%1.1f%%',
        colors=colors, startangle=90)
ax1.set_title('Percentual de Grupos')

ax2.barh(dist_risco['faixa_risco'], dist_risco['bc_total']/1e6, color=colors)
ax2.set_xlabel('Base de Cálculo Total (Milhões R$)')
ax2.set_title('Impacto Financeiro por Faixa')
ax2.invert_yaxis()

plt.tight_layout()
plt.show()

In [ ]:
print("\n" + "=" * 80)
print("2. RANKING: TOP 30 GRUPOS DE MAIOR RISCO")
print("=" * 80)

query_ranking = """
SELECT 
    num_grupo,
    qntd_cnpj,
    score_final_avancado,
    valor_max,
    total_funcionarios,
    nivel_risco_grupo_economico,
    
    -- Componentes individuais do score
    total as score_inconsistencias,
    indice_interconexao,
    indice_risco_grupo_economico,
    indice_risco_indicios,
    indice_risco_pagamentos,
    indice_risco_fat_func
    
FROM gessimples.gei_percent
WHERE score_final_avancado IS NOT NULL
ORDER BY score_final_avancado DESC
LIMIT 30
"""

df_ranking = spark.sql(query_ranking).toPandas()

# Converter Decimal para float
numeric_cols = ['valor_max', 'score_final_avancado', 'indice_interconexao', 
                'indice_risco_grupo_economico', 'indice_risco_indicios',
                'indice_risco_pagamentos', 'indice_risco_fat_func']
for col in numeric_cols:
    if col in df_ranking.columns:
        df_ranking[col] = pd.to_numeric(df_ranking[col], errors='coerce')

print("\nTOP 30 GRUPOS POR SCORE DE RISCO:\n")
for idx, row in df_ranking.iterrows():
    print(f"{idx+1:2d}. Grupo {row['num_grupo']}")
    print(f"    Score Final: {row['score_final_avancado']:.2f} | CNPJs: {int(row['qntd_cnpj'])}")
    print(f"    Receita: R$ {row['valor_max']:,.2f} | Funcionários: {int(row['total_funcionarios'])}")
    print(f"    Nível Risco C115: {row['nivel_risco_grupo_economico'] or 'INEXISTENTE'}")
    print()

# Visualização - Heatmap de componentes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
fig.suptitle('Análise de Risco - Top 30 Grupos', fontsize=16, fontweight='bold')

# Preparar dados para heatmap (Top 15)
risk_components = df_ranking.head(15)[['score_inconsistencias', 'indice_interconexao', 
                                        'indice_risco_grupo_economico', 'indice_risco_indicios',
                                        'indice_risco_pagamentos', 'indice_risco_fat_func']].fillna(0)
risk_components.index = df_ranking.head(15)['num_grupo'].astype(str)

# Normalizar para visualização
risk_components_norm = risk_components.copy()
for col in risk_components_norm.columns:
    col_min = risk_components_norm[col].min()
    col_max = risk_components_norm[col].max()
    if col_max > col_min:
        risk_components_norm[col] = (risk_components_norm[col] - col_min) / (col_max - col_min)
    else:
        risk_components_norm[col] = 0

sns.heatmap(risk_components_norm.T, annot=True, fmt='.2f', cmap='YlOrRd',
            cbar_kws={'label': 'Índice Normalizado (0-1)'}, linewidths=0.5, ax=ax1)
ax1.set_title('Composição do Risco - Top 15 Grupos')
ax1.set_xlabel('Grupo')
ax1.set_ylabel('Componente de Risco')

# Gráfico de barras do score final
top15 = df_ranking.head(15).sort_values('score_final_avancado', ascending=True)
colors_risk = ['#8b0000' if x >= 90 else '#d62728' if x >= 80 else '#ff7f0e' 
               for x in top15['score_final_avancado']]
ax2.barh(top15['num_grupo'].astype(str), top15['score_final_avancado'], color=colors_risk)
ax2.set_xlabel('Score Final de Risco')
ax2.set_ylabel('Grupo')
ax2.set_title('Score Final - Top 15 Grupos')
ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()